In [1]:
import pandas as pd
import numpy as np
import anndata as ann
import scanpy as sc
import squidpy as sq

In [2]:
folder = '/lustre/groups/ml01/workspace/sara.jimenez/spatial_pancreas_data/preprocessed_data/data4downstream/'
S1 = sc.read(folder+'S1_npod_annotation_filtered.h5ad')
S2 = sc.read(folder+'S2_npod_annotation_filtered.h5ad')
S3 = sc.read(folder+'S3_npod_annotation_filtered.h5ad')

/home/aih/sara.jimenez/miniconda3/envs/spatial_v2/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/aih/sara.jimenez/miniconda3/envs/spatial_v2/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/aih/sara.jimenez/miniconda3/envs/spatial_v2/lib/python3.9/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [3]:
adata = sc.concat([S1,S2,S3])
adata

/home/aih/sara.jimenez/miniconda3/envs/spatial_v2/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 386727 × 979
    obs: 'fov', 'Area', 'AspectRatio', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.GCG', 'Max.GCG', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI', 'cell_ID', 'condition', 'slide', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_NegPrb', 'log1p_total_counts_NegPrb', 'pct_counts_NegPrb', 'n_genes', 'cell_type_coarse', 'CellTypes_max'
    obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_fov'
    layers: 'counts'

In [4]:
adata.obs['donor'] = adata.obs[['condition', 'slide']].apply(lambda row: f'{row["condition"]}_{row["slide"]}', axis=1)
adata.obs['sample'] = adata.obs[['condition', 'slide', 'fov']].apply(lambda row: f'{row["condition"]}_{row["slide"]}_{row["fov"]}', axis=1)
adata

AnnData object with n_obs × n_vars = 386727 × 979
    obs: 'fov', 'Area', 'AspectRatio', 'CenterX_global_px', 'CenterY_global_px', 'Width', 'Height', 'Mean.MembraneStain', 'Max.MembraneStain', 'Mean.PanCK', 'Max.PanCK', 'Mean.GCG', 'Max.GCG', 'Mean.CD3', 'Max.CD3', 'Mean.DAPI', 'Max.DAPI', 'cell_ID', 'condition', 'slide', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_NegPrb', 'log1p_total_counts_NegPrb', 'pct_counts_NegPrb', 'n_genes', 'cell_type_coarse', 'CellTypes_max', 'donor', 'sample'
    obsm: 'X_pca', 'X_umap', 'spatial', 'spatial_fov'
    layers: 'counts'

In [5]:
# Step 1: Extract conditions and cell types
conditions = adata.obs['condition']
cell_types = adata.obs['CellTypes_max']

In [22]:
# Step 2: Calculate counts for each condition
counts = adata.obs.groupby(['condition', 'CellTypes_max']).size().unstack(fill_value=0)

# Split counts by conditions
condition1 = counts.loc['ND']  # Replace with your actual condition names
condition2 = counts.loc['T1D']

/tmp/ipykernel_2942651/3009229347.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts = adata.obs.groupby(['condition', 'CellTypes_max']).size().unstack(fill_value=0)


In [23]:
counts

CellTypes_max,Acinar_basal,Acinar_highEnz,Acinar_sigdiff,Acinar_signal,Activated_Stellate,Alpha,Beta,Delta,Ductal,Endothelial,MUC5b_Ductal,Macrophage,Quiescent_Stellate,Tcells
condition,,,,,,,,,,,,,,
ND,16088,27414,27579,13944,7395,8789,7557,3246,21953,7005,4340,5591,6493,4045
T1D,17416,38041,42974,18322,9817,10014,6754,4046,34525,10163,6307,11875,8557,6477


In [24]:
condition1

CellTypes_max
Acinar_basal          16088
Acinar_highEnz        27414
Acinar_sigdiff        27579
Acinar_signal         13944
Activated_Stellate     7395
Alpha                  8789
Beta                   7557
Delta                  3246
Ductal                21953
Endothelial            7005
MUC5b_Ductal           4340
Macrophage             5591
Quiescent_Stellate     6493
Tcells                 4045
Name: ND, dtype: int64

In [25]:
# Step 3: Perform Fisher’s exact test
fisher_results = pd.DataFrame(index=counts.columns, columns=['odds_ratio', 'p_value'])

for cell_type in counts.columns:
    # Create a 2x2 contingency table
    table = [
        [condition1[cell_type], condition2[cell_type]],  # Present in condition1 vs condition2
        [condition1.sum() - condition1[cell_type], condition2.sum() - condition2[cell_type]]  # Not present
    ]
    
    # Perform Fisher's exact test
    odds_ratio, p_value = stats.fisher_exact(table)
    fisher_results.loc[cell_type] = [odds_ratio, p_value]

print(fisher_results)

                   odds_ratio   p_value
CellTypes_max                          
Acinar_basal         1.321088       0.0
Acinar_highEnz       1.006815  0.436391
Acinar_sigdiff       0.874061       0.0
Acinar_signal        1.067914       0.0
Activated_Stellate   1.053667  0.000924
Alpha                1.237732       0.0
Beta                 1.588984       0.0
Delta                1.122026  0.000001
Ductal               0.869608       0.0
Endothelial          0.960139  0.010561
MUC5b_Ductal          0.95918  0.037292
Macrophage           0.644726       0.0
Quiescent_Stellate   1.061364  0.000399
Tcells               0.868212       0.0


### Interpretation of the Odds Ratio:

- Odds Ratio > 1: The cell type is more likely to occur in Condition 1 (ND) compared to Condition 2 (T1D).

- Odds Ratio = 1: The cell type is equally likely to occur in both conditions. There is no association between the cell type and the condition.
  
- Odds Ratio < 1:The cell type is less likely to occur in Condition 1 (ND) compared to Condition 2 (T1D).
